## Ejercicio 1. Creación de una bases de datos de películas extraídas de un API

 Fase 1: Extracción de datos de películas

In [61]:
## Importamos las librerias que vamos a utilizar
import requests      #PAra importar los datos de un API
import pandas as pd   # PAra ver los datos
import mysql.connector #Para conectarnos a MySql
import numpy as np
from mysql.connector import Error

In [62]:
url = "https://beta.adalab.es/resources/apis/pelis/pelis.json"

datos = requests.get(url) #Para que me de los datos del url
print(datos.status_code)  #Si me sale 200 es que esta todo correcto
print(datos.content)  #Imprimo el contenido



200
b'[\n    {\n        "id": 1,\n        "titulo": "The Godfather",\n        "a\xc3\xb1o": 1972,\n        "duracion": 175,\n        "genero": "Crimen",\n        "adultos": false,\n        "subtitulos": [\n            "es",\n            "en"\n        ]\n    },\n    {\n        "id": 2,\n        "titulo": "The Godfather Part II",\n        "a\xc3\xb1o": 1974,\n        "duracion": 202,\n        "genero": "Crimen",\n        "adultos": false,\n        "subtitulos": [\n            "es",\n            "en"\n        ]\n    },\n    {\n        "id": 3,\n        "titulo": "Pulp Fiction",\n        "a\xc3\xb1o": 1994,\n        "duracion": 154,\n        "genero": "Crimen",\n        "adultos": true,\n        "subtitulos": [\n            "es",\n            "en"\n        ]\n    },\n    {\n        "id": 4,\n        "titulo": "Forrest Gump",\n        "a\xc3\xb1o": 1994,\n        "duracion": 142,\n        "genero": "Drama",\n        "adultos": false,\n        "subtitulos": [\n            "es",\n            

In [63]:
datos_peliculas = datos.json() #Lo convierto a json para facilitarme su visualizacion
datos_peliculas

[{'id': 1,
  'titulo': 'The Godfather',
  'año': 1972,
  'duracion': 175,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 2,
  'titulo': 'The Godfather Part II',
  'año': 1974,
  'duracion': 202,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 3,
  'titulo': 'Pulp Fiction',
  'año': 1994,
  'duracion': 154,
  'genero': 'Crimen',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 4,
  'titulo': 'Forrest Gump',
  'año': 1994,
  'duracion': 142,
  'genero': 'Drama',
  'adultos': False,
  'subtitulos': ['es', 'en', 'fr']},
 {'id': 5,
  'titulo': 'The Dark Knight',
  'año': 2008,
  'duracion': 152,
  'genero': 'Acción',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 6,
  'titulo': 'Fight Club',
  'año': 1999,
  'duracion': 139,
  'genero': 'Drama',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 7,
  'titulo': 'Inception',
  'año': 2010,
  'duracion': 148,
  'genero': 'Ciencia ficción',
  'adultos

 Fase 2: Creación de la Base de Datos

In [64]:
## Quiero conectar con MySql, asique hago:
try:
    cnx = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='Mew33uga',
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)


Conexión exitosa


In [65]:
# Creo mi base de datos:
try:
    mycursor = cnx.cursor()
    query = "CREATE DATABASE IF NOT EXISTS peliculas"
    mycursor.execute(query)
    print("Query exitosa")
except:
    print("Error.")

Query exitosa


In [66]:
## Voy a crear la tabla de PELICULAS. Primero defino las columnas que va a tener y que clase de datos va a tener cada una:
mycursor.execute("USE peliculas")
query = '''CREATE TABLE peliculas (
    id INT PRIMARY KEY AUTO_INCREMENT, 
    titulo VARCHAR(255) NOT NULL,
    año YEAR,
    duracion INT,
    genero VARCHAR(100),
    adultos VARCHAR(10)
);'''



In [67]:
mycursor.execute(query)

In [68]:
df_peliculas = pd.DataFrame(datos_peliculas)

In [69]:
df_peliculas

,id,titulo,año,duracion,genero,adultos,subtitulos
0,1,The Godfather,1972,175,Crimen,False,"[es, en]"
1,2,The Godfather Part II,1974,202,Crimen,False,"[es, en]"
2,3,Pulp Fiction,1994,154,Crimen,True,"[es, en]"
3,4,Forrest Gump,1994,142,Drama,False,"[es, en, fr]"
4,5,The Dark Knight,2008,152,Acción,False,"[es, en]"
...,...,...,...,...,...,...,...
95,96,La vita è bella,1997,116,Drama,False,"[es, en, it]"
96,97,Requiem for a Dream,2000,102,Drama,True,"[es, en]"
97,98,Memento,2000,113,Thriller,True,"[es, en]"
98,99,Eternal Sunshine of the Spotless Mind,2004,108,Drama,False,"[es, en]"


In [ ]:
## Vamos a cargar los datos:
query_insert = """
INSERT INTO peliculas (titulo, año, duracion, genero, adultos) 
VALUES (%s, %s, %s, %s, %s)
"""


# Convertir DataFrame a lista de tuplas
#pd.notna(): Verifica si el valor no es NaN/null antes de insertarlo
#iterrows(): Itera sobre las filas del DataFrame

# Reemplazar NaN con None para que se inserten como NULL en la BD
# Reemplazar NaN con None de forma más robusta
df_peliculas_final = df_peliculas[['titulo', 'año', 'duracion', 'genero', 'adultos']] #Pongo esto porque no quiero insertar la columna de subtitulos
df_clean = df_peliculas_final.replace({np.nan: None, 'nan': None, 'NaN': None})
# Convertir DataFrame directamente a lista de tuplas
datos = df_clean.values.tolist()

# Ejecutar inserción
mycursor.executemany(query_insert, datos)
cnx.commit()

print(f"{mycursor.rowcount} registros insertados")

print(f"{mycursor.rowcount} registros insertados")

## Tengo en cuenta que la columna "adulto" la ha cambiado de "true" y "false", donde false es 0 y true es uno 

100 registros insertados
100 registros insertados


Fase 4: Obtener información a partir de los datos.

In [ ]:
# Ya teniendo la base de datos creada, las consultas las vamos a hacer en MYSQL directamente